In [1]:
# Imports for making dataframe
import pandas as pd
import numpy as np

In [2]:
# Import for crawling Goodreads
import requests

In [14]:
# Imports for XML
from xml.etree import ElementTree

In [3]:
# Import for finding texts in HTML
from bs4 import BeautifulSoup

In [4]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [5]:
# Goodreads API Developer key
key = 'mTBsGSOvfBaN0OF25U5g'
secret = 'AkIOvUnGYksPPicdwWfQVStbr61O0YzzrVu1ZUmpGA'

In [ ]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Goodreads login
email = '@uci.edu'
password = ''

In [6]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('amazonData.csv')
amazon

,overall,reviewTime,asin,reviewText
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur..."
1,5.0,"12 27, 2012",0001713353,This was one of my favorites when I was a smal...
2,5.0,"08 15, 2011",0001713353,"So, you think you have problems? Things could ..."
3,5.0,"11 24, 2008",0001713353,My mom bought this for my oldest son. He and ...
4,3.0,"09 18, 2006",0001713353,41 years later:\n\nThe cheese is government ch...
...,...,...,...,...
115439,5.0,"11 27, 1999",0030554179,"""All animals are equal, but some are more equa..."
115440,5.0,"09 9, 1999",0030554179,It's a story about animals who take over a far...
115441,5.0,"07 27, 1999",0030554179,We should not be content to reduce this book t...
115442,5.0,"06 5, 1999",0030554179,George Orwell is the pseudonym of Eric A. Blai...


In [7]:
# Unique set of books to crawl from Goodreads
books = set(amazon.asin)
print(len(books))

3007


In [8]:
### NOTE: COMMENT OUT FOR FULL SET
## TEST SUBSET
test = set()
for i,val in enumerate(books):
    if i == 10:
        break
    test.add(val)
books = test
books

{'0006531458',
 '0007225237',
 '0007225520',
 '0007243766',
 '0007256817',
 '0007339585',
 '0007456964',
 '0020854307',
 '0023672501',
 '0029099110'}

In [11]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [13]:
# Set of books not found on Goodreads:
notfound = set()

In [12]:
# Log in Goodreads
driver = webdriver.Chrome()
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys(email)
driver.find_element_by_name('user[password]').send_keys(password)
driver.find_element_by_name('next').click()                            

In [46]:
# Get most popular reviews for each book (10 if there are more than 10 reviews, or all reviews if less)
for book in books:
    
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        
        print('TRYING BOOK:', book)
        
        # Search up book on Goodreads
        res = requests.get('https://www.goodreads.com/book/isbn/ISBN?format=xml', params={'key': key, 'isbn': book})
        
        # Get XML from API
        root = ElementTree.fromstring(res.text)    
        
        # Find iframe from XML
        iframe = root.find('book').find('reviews_widget').text
        
        # Parse HTML of XML to find link to reviews
        soup = BeautifulSoup(iframe, 'html.parser')
        url = soup.find('iframe')['src']
        
        # Open link to reviews using Selenium
        driver.get(str(url))
        
        # Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        links = driver.find_elements_by_link_text('...more') 
        count = 0
        
        for link in links:
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            count += 1
        
        for i in range(count):
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            time = time.text.strip()
            
            # Only save reviews before 2013
            if int(time[-4:]) < 2013:
                # Use BeautifulSoup to save rating
                rating = s.find('meta', itemprop='ratingValue')
                # Use BeautifulSoup to save text review
                review = s.find('div', itemprop='reviewBody')
                # Add review to dataframe
                df = df.append({'overall':rating.get('content'), 'reviewTime':time, 'asin':book, 
                                'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
    except:
        print('   NOT FOUND')
        notfound.add(book)

TRYING BOOK: 0007225520
https://www.goodreads.com/book/isbn/ISBN?format=xml&key=mTBsGSOvfBaN0OF25U5g&isbn=0007225520
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
TRYING BOOK: 0023672501
https://www.goodreads.com/book/isbn/ISBN?format=xml&key=mTBsGSOvfBaN0OF25U5g&isbn=0023672501
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
TRYING BOOK: 0007339585
https://www.goodreads.com/book/isbn/ISBN?format=xml&key=mTBsGSOvfBaN0OF25U5g&isbn=0007339585
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
TRYING BOOK: 0007456964
https://www.goodreads.com/book/isbn/ISBN?format=xml&key=mTBsGSOvfBaN0OF25U5g&isbn=0007456964
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
TRYING BOOK: 0006531458
https://www.goodreads.com/book/isbn/ISBN?format=xml&key=mTBsGSOvfBaN0OF25U5g&isbn=0006531458
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVING REVIEW
   SAVI

In [47]:
df

,overall,reviewTime,asin,reviewText
0,5,"Feb 04, 2011",0007225520,Single Quote Review:\nIt was all there in Kipl...
1,5,"Nov 12, 2007",0007225520,While it is one of the most beautiful tales of...
2,1,"Aug 08, 2012",0007225520,You know those books that you know from the ve...
3,5,"Dec 01, 2008",0007225520,"Kim served as inspiration for my novel \n""The ..."
4,5,"Oct 02, 2011",0007225520,“There is no sin so great as ignorance. Rememb...
5,5,"Feb 04, 2011",0023672501,Single Quote Review:\nIt was all there in Kipl...
6,5,"Nov 12, 2007",0023672501,While it is one of the most beautiful tales of...
7,1,"Aug 08, 2012",0023672501,You know those books that you know from the ve...
8,5,"Dec 01, 2008",0023672501,"Kim served as inspiration for my novel \n""The ..."
9,5,"Oct 02, 2011",0023672501,“There is no sin so great as ignorance. Rememb...


In [48]:
print(notfound)

{'0007225520', '0023672501', '0007243766', '0007456964', '0006531458', '0007256817', '0020854307', '0007339585', '0029099110', '0007225237'}
